In [1]:
from datasets import load_dataset, load_from_disk
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:

class HMMCustom:
    def __init__(self, n_components, n_observations, startprob=None, transmat=None, emissionprob=None, strategy="viterbi"):

        self.n_components_ = n_components
        self.n_observations_ = n_observations
        self.strategy = strategy

        if startprob is None:
            self.startprob_ = np.zeros(n_components)
        else:
            self.startprob_ = startprob
        if transmat is None:
            self.transmat_ = np.zeros((n_components, n_components))
        else:
            self.transmat_ = transmat
        if emissionprob is None:
            self.emissionprob_ = np.zeros((n_components, n_observations))
        else:
            self.emissionprob_ = emissionprob

    def fit(self, X, y):

        # Get start & transition & emission probs
        for idx, sentence in enumerate(X):
            for i, word in enumerate(sentence):

                # Get start prob
                if i == 0:
                    self.startprob_[y[idx][i]] += 1
                # Get transition prob
                else:
                    self.transmat_[y[idx][i], y[idx][i-1]] += 1

                # Get emission prob
                self.emissionprob_[y[idx][i], word] += 1

        # Get start & transition & emission probs
        for i in range(self.n_components_):
            self.startprob_[i] += 1
            for j in range(self.n_components_):
                self.transmat_[i, j] += 1
            for j in range(self.n_observations_):
                self.emissionprob_[i, j] += 1

        with np.errstate(divide='ignore', invalid='ignore'):
            self.startprob_ /= np.sum(self.startprob_)
            self.emissionprob_ /= ( np.sum(self.emissionprob_, axis=1).reshape(-1, 1) )
            # self.emissionprob_ /= ( np.sum(self.transmat_, axis=1).reshape(-1, 1) + self.startprob_.reshape(-1, 1) )
            self.transmat_ /= np.sum(self.transmat_, axis=1).reshape(-1, 1)
            # self.transmat_ /= np.sum(self.transmat_, axis=1)[:, np.newaxis]
        self.transmat_ = np.nan_to_num(self.transmat_)
        self.emissionprob_ = np.nan_to_num(self.emissionprob_)

        # print(self.startprob_)
        # print()
        # for x in self.transmat_:
        #     print(x)
        # print()
        # for x in self.emissionprob_:
        #     print(x)
        # print()

    def _greedy(self, X):
        log_likelihood, hidden_states = 0, []
        prev_state = None

        for i, word in enumerate(X):
            score = -1
            if i == 0:
                for state in range(self.n_components_):
                    prob = self.startprob_[state] * self.emissionprob_[state][word]
                    if prob > score:
                        score = prob
                        prev_state = state
            else:
                for state in range(self.n_components_):
                    prob = self.transmat_[state][prev_state] * self.emissionprob_[state][word]
                    if prob > score:
                        score = prob
                        prev_state = state

            hidden_states.append(prev_state)

            log_likelihood += score
            # print(score, prev_state)

        return log_likelihood, hidden_states

    def _viterbi(self, X):
        log_likelihood, hidden_states = 0, []

        n_steps = len(X)
        m = np.zeros((self.n_components_, n_steps))
        parent = np.ones((self.n_components_, n_steps), dtype=int)

        for i, word in enumerate(X):
            if i == 0:
                for state in range(self.n_components_):
                    # print(self.startprob_[state], self.emissionprob_[state][word])
                    prob = self.startprob_[state] * self.emissionprob_[state][word]
                    if prob > m[state, i]:
                        parent[state, i] = state
                        m[state, i] = prob
            else:
                for s1 in range(self.n_components_):        # prev state
                    for s2 in range(self.n_components_):    # cur  state
                        # print(self.transmat_[s2, s1], self.emissionprob_[s2, word], m[s1, i-1])
                        prob = self.transmat_[s2, s1] * self.emissionprob_[s2, word] * m[s1, i-1]

                        if prob > m[s2, i]:
                            parent[s2, i] = s1
                            m[s2, i] = prob

        # print()
        # for x in m:
        #     print(x)
        # print()
        # for x in parent:
        #     print(x)

        mostLikelyStateIdx = np.argmax(m[:, -1])
        hidden_states.append(mostLikelyStateIdx)
        log_likelihood += m[mostLikelyStateIdx, -1]
        i = n_steps - 2

        while i > 0:
            # put parent of state i
            hidden_states.append(parent[mostLikelyStateIdx, i+1])
            # add likelihood of state i
            log_likelihood += m[parent[mostLikelyStateIdx, i+1], i]
            mostLikelyStateIdx = hidden_states[-1]
            i -= 1

        hidden_states.append(parent[mostLikelyStateIdx, 1])
        log_likelihood += m[hidden_states[-1], 0]


        return log_likelihood, reversed(hidden_states)

    def decode(self, X):

        if self.strategy == "viterbi":
            return self._viterbi(X)
        elif self.strategy == "greedy":
            return self._greedy(X)

In [3]:
dataset = load_dataset("lhoestq/conll2003")
dataset.save_to_disk("conll2003")

dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/281k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14041 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3250 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
random_state = 42

nerTags = dataset["train"][:]['ner_tags']
tokens = dataset["train"][:]['tokens']
states = ["Other", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC"]
observations = set()
maxLen = 0
observations.add(" ")
for token in tokens:
    maxLen = max(maxLen, len(token))
    for word in token:
        observations.add(word.lower())
observations = list(sorted(observations))
# print("Observations:", observations)

le = LabelEncoder()
encoded_data = le.fit_transform(observations)
# print("Encoded data:", encoded_data)

X = []
for token in tokens:
    encoded_tokens = le.transform([word.lower() for word in token])
    X.append(encoded_tokens.tolist())

# print(X)
# print(tokens)
# print(nerTags)

n_components = len(states)
n_observations = len(observations)
# print(n_components)
# print(n_observations)

model = HMMCustom(n_components=n_components, n_observations=n_observations)

model.fit(X, nerTags)

In [6]:
n_samples = 5


y_val = dataset["validation"][:n_samples]['ner_tags']
X_val_tmp = dataset["validation"][:n_samples]['tokens']

# print("y val:", y_val)

X_val = []
dropped_words = []
for token in X_val_tmp:
    lowercaseToken = []
    for word in token:
        word = word.lower()
        if word in le.classes_:
            lowercaseToken.append(word)
        else:
            dropped_words.append(word)
    encoded_tokens = le.transform(lowercaseToken)
    X_val.append(encoded_tokens.tolist())
    
# print("X val:", X_val)
print("Dropped words:", dropped_words, end="\n\n")

for i, sample in enumerate(X_val):
    log_likelihood, hidden_states = model.decode(sample)
    print("Sample", i+1)
    print("Log-likelihood of observations:", log_likelihood)
    print("Most likely sequence:", [states[s] for s in hidden_states])
    print("Observation sequence:", [states[s] for s in y_val[i]])
    
    print()

Dropped words: ['296', 'discard']

Sample 1
Log-likelihood of observations: 0.00022969218062895774
Most likely sequence: ['Other', 'Other', 'B-ORG', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other']
Observation sequence: ['Other', 'Other', 'B-ORG', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other']

Sample 2
Log-likelihood of observations: 0.0006200132375254864
Most likely sequence: ['B-LOC', 'I-LOC']
Observation sequence: ['B-LOC', 'Other']

Sample 3
Log-likelihood of observations: 4.876794444141301e-05
Most likely sequence: ['Other', 'B-MISC', 'Other', 'B-PER', 'I-PER', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'B-ORG', 'Other', 'B-ORG', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other']
Observation sequence: ['B-MISC', 'I-MISC', 'Other', 'B-PER', 'I-PER', 'Other', 'Other', 'Other', 'Other', 'O